# Animating 60 years of US energy data in less than 10 lines of Python

In this step-by-step tutorial, we're going to walk through animating ~60 years of United States energy consumption data with 10 lines of Python! 

The dataset we will be working with is publically available on the [US Energy Information Administration](https://www.eia.gov/) government website. 

We're going to process the data in `pandas`, visualize the data with `plotly.express`, and gain some necessary domain knowledge along the way so we can understand what we're working with

**Prerequisites:**
- [pandas](https://pandas.pydata.org/)
- [plotly](https://plotly.com/python/)

# Importing the libraries

In [1]:
import pandas as pd 
import plotly.express as px 

# Importing the dataset 

In [2]:
df = pd.read_csv("https://www.eia.gov/state/seds/sep_use/total/csv/use_all_btu.csv")
df


,Data_Status,State,MSN,1960,1961,1962,1963,1964,1965,1966,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,2018F,AK,ABICB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-14.0,-4.0,0.0,0.0,-5.0,-2.0,-3.0,-3.0,-2.0,-14.0
1,2018F,AK,ARICB,312.0,555.0,489.0,589.0,791.0,878.0,1646.0,...,10198.0,12644.0,14931.0,14343.0,12763.0,12335.0,12450.0,12365.0,13561.0,4804.0
2,2018F,AK,ARTCB,312.0,555.0,489.0,589.0,791.0,878.0,1646.0,...,10198.0,12644.0,14931.0,14343.0,12763.0,12335.0,12450.0,12365.0,13561.0,4804.0
3,2018F,AK,ARTXB,312.0,555.0,489.0,589.0,791.0,878.0,1646.0,...,10198.0,12644.0,14931.0,14343.0,12763.0,12335.0,12450.0,12365.0,13561.0,4804.0
4,2018F,AK,AVACB,5209.0,6441.0,5202.0,3273.0,2319.0,1478.0,1086.0,...,1095.0,854.0,801.0,776.0,701.0,658.0,1308.0,1240.0,1239.0,1264.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10970,2018F,WY,WYCCB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10971,2018F,WY,WYEGB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21728.0,31676.0,44809.0,41576.0,42298.0,41899.0,35009.0,40522.0,39806.0,36936.0
10972,2018F,WY,WYICB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10973,2018F,WY,WYTCB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21728.0,31676.0,44809.0,41576.0,42298.0,41899.0,35009.0,40522.0,39806.0,36936.0


# Removing unnecessary data

In this case, the data we have contains rows for both the United States as a whole and Washington DC. Since we are concerned with only the states, we're just going to filter these rows out.

In [3]:
df = df[~df.isin(["US", "DC"])]

# Understanding the MSN column

Looking through [documentation](https://www.eia.gov/state/seds/sep_fuel/html/csv/fuel_csv_doc.pdf) provided on the website, we can see that the `"MSN"` column stands for Mnemonic Series Names. These are codes that contain information on the type of energy source, the sector, and the unit. 

For example, row 0's MSN value is _ABICB_.

Let's break it down:
- _AB_: Aviation gasoline blending components 
- _IC_: Industrial sector 
- _B_: British thermal units (BTU)

Thus, that row corresponds to _Aviation gasoline blending components consumed by the industrial sector in british thermal units (BTU)_.

# Getting total across all sectors per energy source 

When the 3rd and 4th characters of an  _MSN_ are _TC_, this means the row corresponds to the total energy consumption across all sectors for that resource.

Looking back at the Aviation gasoline blending components, an _MSN_ of _ABTCB_ would be 
- _AB_: Aviation gasoline blending components 
- _TC_: Total of all sectors
- _B_: British thermal units (BTU)

To get all rows that have _TC_ as the 3rd and 4th characters, we can use a regular expression and the `Series.str.match` method. 

In [14]:
total_df = df[df["MSN"].str.match("[A-Z]{2}TC[A-Z]")]
total_df["MSN"].unique()

array(['ARTCB', 'AVTCB', 'BDTCB', 'BFTCB', 'BMTCB', 'BQTCB', 'BYTCB',
       'CLTCB', 'DFTCB', 'DMTCB', 'EMTCB', 'EQTCB', 'ESTCB', 'EYTCB',
       'FFTCB', 'GETCB', 'HLTCB', 'HYTCB', 'IQTCB', 'IYTCB', 'JFTCB',
       'KSTCB', 'LOTCB', 'LUTCB', 'MGTCB', 'MMTCB', 'NGTCB', 'NNTCB',
       'OPTCB', 'PATCB', 'PCTCB', 'PMTCB', 'PPTCB', 'PQTCB', 'PYTCB',
       'RETCB', 'RFTCB', 'SFTCB', 'SOTCB', 'TETCB', 'WDTCB', 'WSTCB',
       'WWTCB', 'WYTCB'], dtype=object)

# Melting years into a Year column

Now that we have the rows we want to visualize, we're going to unpivot our `DataFrame` from wide to long format. This will massage the year columns into rows as two new columns, `"Year"` and `"BTU"`.

This allows us to select and filter our data much easier now as well as pass it through to `plotly.express` which is expecting a long format `DataFrame`.

In [5]:
total_df = total_df.melt(id_vars=["Data_Status", "State", "MSN"], var_name="Year", value_name="BTU")
total_df

,Data_Status,State,MSN,Year,BTU
0,2018F,AK,ARTCB,1960,312.0
1,2018F,AK,AVTCB,1960,5209.0
2,2018F,AK,BDTCB,1960,0.0
3,2018F,AK,BFTCB,1960,0.0
4,2018F,AK,BMTCB,1960,3681.0
...,...,...,...,...,...
134987,2018F,WY,TETCB,2018,558594.0
134988,2018F,WY,WDTCB,2018,4899.0
134989,2018F,WY,WSTCB,2018,2.0
134990,2018F,WY,WWTCB,2018,4901.0


# Summing all energy sources together

Now that we have the total consumption per energy source, state, and year, we can sum them all together grouped by each state and each year. This will leave us with the total energy consumption per state per year which is what we're looking to visualize. 

In [6]:
summed_df = total_df.groupby(["State", "Year"], as_index=False).sum()
summed_df

,State,Year,BTU
0,AK,1960,313183.0
1,AK,1961,360673.0
2,AK,1962,389417.0
3,AK,1963,403120.0
4,AK,1964,427825.0
...,...,...,...
2945,WY,2014,3090675.0
2946,WY,2015,2945078.0
2947,WY,2016,2886446.0
2948,WY,2017,3010681.0


# Creating the animated bar plot

Our data is now ready to be visualized! `plotly.express.bar` has an `animation_frame` argument which allows us to choose what column to animate our data on. For this example, we will be animating one frame per year of data. 

In [9]:
fig = px.bar(
        summed_df, 
        x="State", 
        y="BTU", 
        animation_frame="Year", 
        range_y=(
            0, 
            summed_df["BTU"].max()
        ), 
        color="State",
        title="United States total energy consumption (BTU)"
).update_xaxes(categoryorder="total ascending")

fig.show()